In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import hmean
from pingouin import wilcoxon

sns.set_theme(style="whitegrid")

In [39]:
def compute_statistics(col_name, table_col, condition):
  for data in baseline_df['data'].unique():
    print(data)
    baseline_acc = baseline_df[baseline_df['data'] == data][col_name]
    for constraint in df_geg2['constraint'].unique():
      geg_acc = df_geg2[(df_geg2['data'] == data) & (df_geg2['constraint'] == constraint)][col_name]
      stat = wilcoxon(baseline_acc.abs(), geg_acc.abs(), alternative=condition)
      if stat['p-val'].values[0] <= 0.05:
        table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col] = table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col].apply(lambda x: '\isBetter '+ x)
      elif stat['p-val'].values[0] > 0.99:
        table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col] = table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col].apply(lambda x: '\isWorse '+ x)

## RQ1

In [17]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_baseline'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_baseline', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [18]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [20]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_10485/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [21]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_new'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_new', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [22]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_10485/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [24]:
grouped_baseline['constraint'] = 'baseline'

In [35]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [36]:
full_data.head()

data  accuracy           precision              recall            \
                mean       std      mean       std      mean       std   
0      cmc  0.605585  0.032657  0.583410  0.037187  0.559529  0.032210   
1    crime  0.474377  0.036656  0.433933  0.045527  0.452207  0.032034   
2     drug  0.686975  0.023164  0.618495  0.032670  0.613888  0.018050   
3      law  0.665588  0.015001  0.640340  0.015826  0.652344  0.014110   
4  obesity  0.667785  0.044095  0.653654  0.046160  0.665343  0.032948   

   f1_score           statistical_parity           equal_opportunity  \
       mean       std               mean       std              mean   
0  0.553113  0.033201           0.114622  0.066492          0.177527   
1  0.427129  0.036688           0.382143  0.061563          0.266450   
2  0.610658  0.025472           0.213459  0.124928          0.275713   
3  0.643703  0.014925           0.082878  0.018551          0.039229   
4  0.651001  0.037760           0.048876  0.043375          0.011934   

            average_odds           constraint  
        std         mean       std             
0  0.128642     0.115679  0.075572   baseline  
1  0.287731     0.246728  0.146296   baseline  
2  0.181188     0.177260  0.099579   baseline  
3  0.047822     0.050955  0.025452   baseline  
4  0.118706     0.011306  0.067495   baseline

In [42]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [47]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)

In [64]:
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_10485/1130035862.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [65]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,cmc,baseline,0.606$\pm$0.033,0.583$\pm$0.037,0.56$\pm$0.032,0.553$\pm$0.033,0.115$\pm$0.066,0.178$\pm$0.129,0.116$\pm$0.076
7,cmc,cp,0.606$\pm$0.03,0.576$\pm$0.038,0.561$\pm$0.032,0.556$\pm$0.034,0.058$\pm$0.062,0.057$\pm$0.162,0.042$\pm$0.087
8,cmc,dp,0.602$\pm$0.028,0.575$\pm$0.032,0.556$\pm$0.026,0.549$\pm$0.027,0.015$\pm$0.057,-0.045$\pm$0.147,-0.02$\pm$0.063
9,cmc,eo,0.601$\pm$0.034,0.574$\pm$0.038,0.56$\pm$0.031,0.557$\pm$0.032,0.028$\pm$0.045,0.01$\pm$0.171,0.007$\pm$0.076
1,crime,baseline,0.474$\pm$0.037,0.434$\pm$0.046,0.452$\pm$0.032,0.427$\pm$0.037,0.382$\pm$0.062,0.266$\pm$0.288,0.247$\pm$0.146
10,crime,cp,0.355$\pm$0.031,0.373$\pm$0.031,0.333$\pm$0.029,0.315$\pm$0.033,0.087$\pm$0.065,0.118$\pm$0.213,0.06$\pm$0.103
11,crime,dp,0.406$\pm$0.04,0.403$\pm$0.035,0.397$\pm$0.038,0.392$\pm$0.035,0.028$\pm$0.048,-0.056$\pm$0.174,-0.072$\pm$0.07
12,crime,eo,0.333$\pm$0.077,0.41$\pm$0.107,0.307$\pm$0.081,0.246$\pm$0.11,0.128$\pm$0.111,0.074$\pm$0.234,0.066$\pm$0.136
2,drug,baseline,0.687$\pm$0.023,0.618$\pm$0.033,0.614$\pm$0.018,0.611$\pm$0.025,0.213$\pm$0.125,0.276$\pm$0.181,0.177$\pm$0.1


In [69]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park


In [71]:
table_tex.to_latex('results_summary.tex', index=False, escape=False)

## RQ2 

In [2]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_baseline_rq2'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_baseline_rq2', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [3]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [4]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_99276/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [5]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_rq2'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_rq2', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [6]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_99276/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [7]:
grouped_baseline['constraint'] = 'baseline'

In [8]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [9]:
full_data.head()

data  accuracy           precision              recall            \
               mean       std      mean       std      mean       std   
0   adult  0.826665  0.006915  0.771957  0.012435  0.726586  0.009579   
1  compas  0.675209  0.021380  0.675343  0.019898  0.666093  0.019814   
2  german  0.745000  0.044284  0.692620  0.063216  0.661321  0.057729   
3   adult  0.769134  0.004880  0.738693  0.024915  0.535491  0.007229   
4   adult  0.824596  0.005983  0.774139  0.012452  0.710732  0.007729   

   f1_score           statistical_parity           equal_opportunity  \
       mean       std               mean       std              mean   
0  0.743824  0.009620          -0.179410  0.015724         -0.154284   
1  0.665794  0.021166          -0.173967  0.040158         -0.102123   
2  0.668692  0.059542          -0.209581  0.107955         -0.182181   
3  0.506410  0.012765          -0.008359  0.007381          0.012244   
4  0.732296  0.008604          -0.076991  0.015889          0.129999   

            average_odds           constraint  
        std         mean       std             
0  0.047611    -0.119063  0.025816   baseline  
1  0.030810    -0.149917  0.041596   baseline  
2  0.170483    -0.169542  0.129837   baseline  
3  0.030731     0.008721  0.016611         cp  
4  0.070503     0.058032  0.036233         dp

In [10]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [11]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_99276/93204417.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [13]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

german
compas
adult
german
compas
adult
german
compas
adult
german
compas
adult
german
compas
adult
german
compas
adult


In [14]:
table_tex.to_latex('results_summary_rq2.tex', index=False, escape=False)

## RQ4

In [40]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_models'):
    if file.endswith('.csv') and '_rf' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_models', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [41]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [42]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [43]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_rq4'):
    if file.endswith('.csv') and '_rf' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_rq4', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [44]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [45]:
grouped_baseline['constraint'] = 'baseline'

In [46]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [47]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [48]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)

In [49]:
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/1130035862.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [50]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,cmc,baseline,0.98$\pm$0.015,0.979$\pm$0.016,0.977$\pm$0.017,0.978$\pm$0.017,0.136$\pm$0.08,0.022$\pm$0.087,0.012$\pm$0.045
7,cmc,cp,0.794$\pm$0.041,0.833$\pm$0.022,0.833$\pm$0.032,0.79$\pm$0.043,0.04$\pm$0.16,0.01$\pm$0.062,-0.036$\pm$0.083
8,cmc,dp,0.868$\pm$0.022,0.869$\pm$0.02,0.882$\pm$0.02,0.86$\pm$0.025,0.029$\pm$0.092,0.015$\pm$0.083,-0.058$\pm$0.068
9,cmc,eo,0.984$\pm$0.009,0.985$\pm$0.008,0.981$\pm$0.01,0.983$\pm$0.009,0.141$\pm$0.084,0.034$\pm$0.078,0.019$\pm$0.04
1,crime,baseline,0.499$\pm$0.031,0.471$\pm$0.03,0.478$\pm$0.025,0.466$\pm$0.027,0.413$\pm$0.069,0.483$\pm$0.232,0.355$\pm$0.14
10,crime,cp,0.415$\pm$0.04,0.442$\pm$0.032,0.386$\pm$0.03,0.364$\pm$0.036,0.19$\pm$0.046,0.379$\pm$0.226,0.197$\pm$0.113
11,crime,dp,0.437$\pm$0.04,0.441$\pm$0.033,0.411$\pm$0.022,0.395$\pm$0.03,0.155$\pm$0.055,0.307$\pm$0.244,0.139$\pm$0.124
12,crime,eo,0.507$\pm$0.036,0.473$\pm$0.036,0.484$\pm$0.029,0.47$\pm$0.034,0.424$\pm$0.061,0.497$\pm$0.194,0.367$\pm$0.117
2,drug,baseline,0.676$\pm$0.032,0.606$\pm$0.033,0.597$\pm$0.027,0.597$\pm$0.028,0.143$\pm$0.125,0.177$\pm$0.194,0.108$\pm$0.128


In [51]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime


In [53]:
table_tex.to_latex('results_summary_rq4_rf.tex', index=False, escape=False)

## SVM

In [54]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_models'):
    if file.endswith('.csv') and '_svm' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_models', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [55]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [56]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [57]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_rq4'):
    if file.endswith('.csv') and '_svm' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_rq4', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [58]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [59]:
grouped_baseline['constraint'] = 'baseline'

In [60]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [61]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [62]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)

In [63]:
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/1130035862.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [64]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,cmc,baseline,0.777$\pm$0.039,0.835$\pm$0.028,0.777$\pm$0.029,0.773$\pm$0.036,0.076$\pm$0.089,0.031$\pm$0.242,0.008$\pm$0.12
7,cmc,cp,0.646$\pm$0.072,0.696$\pm$0.051,0.672$\pm$0.056,0.64$\pm$0.068,0.036$\pm$0.121,0.005$\pm$0.197,-0.02$\pm$0.128
8,cmc,dp,0.273$\pm$0.039,0.61$\pm$0.109,0.372$\pm$0.014,0.211$\pm$0.029,0.027$\pm$0.046,0.02$\pm$0.047,0.022$\pm$0.038
9,cmc,eo,0.484$\pm$0.187,0.665$\pm$0.081,0.542$\pm$0.146,0.468$\pm$0.2,0.052$\pm$0.09,0.026$\pm$0.088,0.017$\pm$0.077
1,crime,baseline,0.234$\pm$0.042,0.063$\pm$0.03,0.198$\pm$0.011,0.086$\pm$0.025,0.001$\pm$0.014,-0.016$\pm$0.063,-0.006$\pm$0.032
10,crime,cp,0.233$\pm$0.038,0.067$\pm$0.036,0.199$\pm$0.008,0.086$\pm$0.024,-0.002$\pm$0.032,-0.012$\pm$0.054,-0.007$\pm$0.037
11,crime,dp,0.235$\pm$0.044,0.065$\pm$0.031,0.199$\pm$0.011,0.087$\pm$0.025,0.006$\pm$0.016,-0.014$\pm$0.062,-0.004$\pm$0.029
12,crime,eo,0.235$\pm$0.034,0.073$\pm$0.059,0.201$\pm$0.004,0.086$\pm$0.03,-0.005$\pm$0.017,-0.02$\pm$0.057,-0.011$\pm$0.031
2,drug,baseline,0.501$\pm$0.034,0.466$\pm$0.049,0.466$\pm$0.047,0.441$\pm$0.045,0.045$\pm$0.111,-0.016$\pm$0.304,0.001$\pm$0.153


In [65]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity


/Users/giord/miniconda3/envs/ds/lib/python3.10/site-packages/scipy/stats/_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
/Users/giord/miniconda3/envs/ds/lib/python3.10/site-packages/pingouin/nonparametric.py:493: RuntimeWarning: invalid value encountered in scalar divide
  rbc = r_plus / rsum - r_minus / rsum


drug
crime
cmc
law
park
wine
obesity
drug
crime


In [66]:
table_tex.to_latex('results_summary_rq4_svm.tex', index=False, escape=False)